# Justin Chu.  Pandas Practice Sheet.  Displaying skills in pandas such as retrieving, manipulating and transfroming data frames.

### The descriptions following the exercise titles are simply notes for myself to help remember the syntax.  They may not make sense to you but they make sense to me :)

# Importing libraries

In [190]:
import pandas as pd
import pandas.util.testing
import numpy as np
import os
from requests import request

# Turning a wide DF into a long DF (df.melt(id_vars=, var_name=, value_name=)

In [191]:

d = {\
"zip_code": [12345, 56789, 101112, 131415],
"factory": [100, 400, 500, 600],
"warehouse": [200, 300, 400, 500],
"retail": [1, 2, 3, 4]
}

df = pd.DataFrame(d)
df

,zip_code,factory,warehouse,retail
0,12345,100,200,1
1,56789,400,300,2
2,101112,500,400,3
3,131415,600,500,4


In [192]:
df = df.melt(id_vars='zip_code', var_name='Location', value_name='Distance')
df

,zip_code,Location,Distance
0,12345,factory,100
1,56789,factory,400
2,101112,factory,500
3,131415,factory,600
4,12345,warehouse,200
5,56789,warehouse,300
6,101112,warehouse,400
7,131415,warehouse,500
8,12345,retail,1
9,56789,retail,2


# Converting Year and Day of year into a single df column. Combine year and day to a single number, ex. 2019340, then use pd.to_datetime(year&day, format= '%Y%j')

In [193]:
d = {\
"year": [2019, 2019, 2020],
"day_of_year": [350, 365, 1]
}

df = pd.DataFrame(d)
df

,year,day_of_year
0,2019,350
1,2019,365
2,2020,1


In [194]:
# creates combined column of the year pluse the day of year, ex: 2019350.  The *1000 is to make the number large 
# enough to contain the days after the year. 
df['combined'] = df['year']*1000 + df['day_of_year']

#convert to a date time:
df['Date'] = pd.to_datetime(df['combined'], format='%Y%j')
#drop the old columns 
df.drop(columns=(['year', 'day_of_year', 'combined']), inplace=True)

In [195]:
df

,Date
0,2019-12-16
1,2019-12-31
2,2020-01-01


# How to count null or nan values. Combine isnull() or isna() with sum().

In [196]:
df.isnull().sum().sum()
#OR
df.isna().sum()

Date    0
dtype: int64

In [197]:
d = {"customer": ["A", "B", "C", "D"], "sales":[1100, 950.75, "$400", "$1250.35"]}
df = pd.DataFrame(d)
df

,customer,sales
0,A,1100
1,B,950.75
2,C,$400
3,D,$1250.35


# Cleaning a column using regex.  Remember to encapsulate the string in brackets [ ].

In [198]:
df['sales'] = df['sales'].replace('[$]' , '', regex=True).astype('float')

In [199]:
df

,customer,sales
0,A,1100.00
1,B,950.75
2,C,400.00
3,D,1250.35


# How to make a testing timeseries df.  pd.util.testing.makeTimeDataFrame or pd.util.testing.makeDateIndex 

In [200]:
rows = 365*24 #number of hours in a year
pd.util.testing.makeTimeDataFrame(rows, freq='H')

,A,B,C,D
2000-01-01 00:00:00,0.536348,0.556695,0.111654,-1.518898
2000-01-01 01:00:00,-1.861562,0.848101,-0.262202,-0.289345
2000-01-01 02:00:00,1.414418,2.383990,1.100132,-0.167113
2000-01-01 03:00:00,-0.126185,0.636716,1.699359,0.476539
2000-01-01 04:00:00,-2.136134,0.701303,0.907565,-0.432365
...,...,...,...,...
2000-12-30 19:00:00,-0.167623,0.420766,0.308602,-0.011937
2000-12-30 20:00:00,-0.599863,0.445246,-0.971073,-0.018881
2000-12-30 21:00:00,0.501984,0.343041,0.578630,-0.513265
2000-12-30 22:00:00,0.512649,0.413180,-0.071745,0.529722


In [201]:
#OR 
col = 2 
col_names = ['sales', 'customers']
df = pd.DataFrame(np.random.randint(1,20, size=(rows, col)), columns=col_names)
df.index = pd.util.testing.makeDateIndex(rows, freq='H')
df

,sales,customers
2000-01-01 00:00:00,13,6
2000-01-01 01:00:00,5,8
2000-01-01 02:00:00,13,5
2000-01-01 03:00:00,12,14
2000-01-01 04:00:00,7,1
...,...,...
2000-12-30 19:00:00,17,18
2000-12-30 20:00:00,5,1
2000-12-30 21:00:00,14,16
2000-12-30 22:00:00,10,14


# How to move columns to a specific location

In [202]:
d = {"A":[15, 20], "B":[20, 25], "C":[30 ,40], "D":[50, 60]}
df = pd.DataFrame(d)
df


,A,B,C,D
0,15,20,30,50
1,20,25,40,60


In [203]:
df.insert(3, 'After C', df['C']*3)


In [204]:
df

,A,B,C,After C,D
0,15,20,30,90,50
1,20,25,40,120,60


# How to split names into first name, last name columns. split(n=?) n is the number of maximum splits to perform.  In this case we only want one split.  Note the row data must be accessed as a string. 

In [205]:
df = pd.Series(["Geordi La Forge", "Deanna Troi", "Data"]).to_frame()
df.rename({0: 'Name'}, axis=1, inplace=True)


In [206]:
df['First Name'] = df['Name'].str.split(n=1).str[0]
df['Last Name'] = df['Name'].str.split(n=1).str[1]

In [207]:
df

,Name,First Name,Last Name
0,Geordi La Forge,Geordi,La Forge
1,Deanna Troi,Deanna,Troi
2,Data,Data,NaN


# Rearrange column names. Simply set the df to itself in the order you want.  Note the use of double brackets: the index and the list in itself.

In [208]:
df = df[['First Name', 'Last Name', 'Name']]
df

,First Name,Last Name,Name
0,Geordi,La Forge,Geordi La Forge
1,Deanna,Troi,Deanna Troi
2,Data,NaN,Data


# Aggregate datetimes by day and filter weekends

In [209]:
col = 2 
col_names = ['sales', 'customers']
df = pd.DataFrame(np.random.randint(1,20, size=(rows, col)), columns=col_names)
df.index = pd.util.testing.makeDateIndex(rows, freq='H')
df
df

,sales,customers
2000-01-01 00:00:00,2,14
2000-01-01 01:00:00,8,3
2000-01-01 02:00:00,18,16
2000-01-01 03:00:00,3,3
2000-01-01 04:00:00,11,16
...,...,...
2000-12-30 19:00:00,3,3
2000-12-30 20:00:00,8,18
2000-12-30 21:00:00,4,3
2000-12-30 22:00:00,14,19


In [210]:
df.reset_index(inplace=True)


In [211]:
df.rename(columns={'index': 'Date'}, inplace=True)
df['Day'] = df['Date'].dt.day_name()

In [212]:
df_weekend = df[(df['Day'] == 'Saturday') | (df['Day'] == 'Sunday')]
df_weekend['Day'].unique()

array(['Saturday', 'Sunday'], dtype=object)

In [213]:
df_week_day = df[(df['Day'] != 'Saturday') & (df['Day'] != 'Sunday')]

In [214]:
df_week_day['Day'].unique()

array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
      dtype=object)

In [215]:
groups = df.groupby(['Day']).mean()
groups

,sales,customers
Day,,
Friday,10.332532,9.667468
Monday,9.845353,9.904647
Saturday,10.000000,10.055818
Sunday,9.928686,9.891026
Thursday,9.927083,9.827724
Tuesday,10.032051,10.175481
Wednesday,9.879808,9.874199


In [216]:
weekend_sales = df_weekend['sales'].sum()
weekend_sales

25111

In [217]:
week_sales = df_week_day['sales'].sum()
week_sales

62421

# Converting values into another type of values

In [218]:
d = {"gender":["male", "female", "male"], "color":["red", "green", "blue"], "age":[25, 30, 15]}
df = pd.DataFrame(d)
df

,gender,color,age
0,male,red,25
1,female,green,30
2,male,blue,15


In [219]:
gender_dict = {'male': 'M', 'female': 'F'}
df['GENDER'] = df['gender'].map(gender_dict)
df['Color Code'] = df['color'].factorize()[0] #factorize returns (array[0,1,2], index['red', 'green', 'blue'], 
                                              # dtype='object') so we select index 0.
df['Adult'] = df['age'] > 18

In [220]:
df

,gender,color,age,GENDER,Color Code,Adult
0,male,red,25,M,0,True
1,female,green,30,F,1,True
2,male,blue,15,M,2,False
